## Import Libraries

In [1]:
from itertools import product

In [2]:
singularity_cmd = 'singularity exec --nv --overlay /scratch/tm3647/public/mlh-env-50G-10M.ext3:ro --overlay /scratch/tm3647/public/CheXpert_v1.sqf:ro --overlay /scratch/tm3647/public/Mimic_cxr_jpg.sqf:ro /scratch/work/public/singularity/cuda11.3.0-cudnn8-devel-ubuntu20.04.sif /bin/bash -c "'

## General Hyperparamters

In [3]:
num_rounds = [100]
num_clients = [4, 8, 16]
num_local_epochs = [1]

In [4]:
general_hyperparams = list([num_rounds, num_clients, num_local_epochs])
general_hyperparams

[[100], [4, 8, 16], [1]]

In [6]:
list(product(*general_hyperparams))

[(100, 4, 1), (100, 8, 1), (100, 16, 1)]

## Volume Configs

In [7]:
scales = [1, 10, 100, 1000]
modes = ['dirchlet']

In [8]:
volume_hyperparams = general_hyperparams+list([scales, modes])

In [9]:
volume_hyperparams

[[100], [4, 8, 16], [1], [1, 10, 100, 1000], ['dirchlet']]

In [11]:
total_exp_combinations = list(product(*volume_hyperparams))
len(total_exp_combinations), total_exp_combinations[0]

(12, (100, 4, 1, 1, 'dirchlet'))

In [12]:
env_activate_cmd = """
source /ext3/miniconda3/etc/profile.d/conda.sh;

conda activate /ext3/fedml-env/;

cd /scratch/tm3647/MLH/ConVIRT-private/;
"""

In [16]:
for nr, nc, num_l_ep, scale_val, mode_val in total_exp_combinations:
    job_name = f"fed_vol_{nr}_{nc}_{scale_val}_{mode_val}"
    sbatch_headers = f"""#!/bin/bash
    
#SBATCH --nodes=1
#SBATCH --cpus-per-task=28
#SBATCH --time=3:00:00
#SBATCH --mem=128GB
#SBATCH --gres=gpu:a100:1
#SBATCH --job-name={job_name}
#SBATCH --output=/scratch/tm3647/MLH/ConVIRT-private/sbatch_outs/%j_0_log.out
#SBATCH --mail-type=ALL
#SBATCH --mail-user=tm3647@nyu.edu

    """
    
    py_cmd = f"\npython src/run_simulation.py --config-name=prod_simulation server_config.num_rounds={nr} pool_size={nc} partitions.num_clients={nc} partitions.scale={scale_val} partitions.mode='{mode_val}' task_name='fed_chexpert_volume'"
    
    sbatch_script = sbatch_headers + singularity_cmd + env_activate_cmd + py_cmd + "\n\""
    
    with open(f'/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/{job_name}.sbatch', 'w') as f:
        f.write(sbatch_script)

In [17]:
!cat "/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_4_1_dirchlet.sbatch"

#!/bin/bash
    
#SBATCH --nodes=1
#SBATCH --cpus-per-task=28
#SBATCH --time=3:00:00
#SBATCH --mem=128GB
#SBATCH --gres=gpu:a100:1
#SBATCH --job-name=fed_vol_100_4_1_dirchlet
#SBATCH --output=/scratch/tm3647/MLH/ConVIRT-private/sbatch_outs/%j_0_log.out
#SBATCH --mail-type=ALL
#SBATCH --mail-user=tm3647@nyu.edu

    singularity exec --nv --overlay /scratch/tm3647/public/mlh-env-50G-10M.ext3:ro --overlay /scratch/tm3647/public/CheXpert_v1.sqf:ro --overlay /scratch/tm3647/public/Mimic_cxr_jpg.sqf:ro /scratch/work/public/singularity/cuda11.3.0-cudnn8-devel-ubuntu20.04.sif /bin/bash -c "
source /ext3/miniconda3/etc/profile.d/conda.sh;

conda activate /ext3/fedml-env/;

cd /scratch/tm3647/MLH/ConVIRT-private/;

python src/run_simulation.py --config-name=prod_simulation server_config.num_rounds=100 pool_size=4 partitions.num_clients=4 partitions.scale=1 partitions.mode='dirchlet' task_name='fed_chexpert_volume'
"

In [18]:
import glob

In [19]:
files = glob.glob('/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/*.sbatch')

In [20]:
files

['/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_8_100_dirchlet.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_4_100_dirchlet.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_16_1000_dirchlet.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_16_100_dirchlet.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_8_10_dirchlet.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_4_1000_dirchlet.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_16_1_dirchlet.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_4_10_dirchlet.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_16_10_dirchlet.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/vo

In [30]:
sbatch_execs = ""
for f in files:
    sbatch_execs += f"sbatch {f}\nsleep 2\n"

In [31]:
print(sbatch_execs)

sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_8_100_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_4_100_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_16_1000_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_16_100_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_8_10_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_4_1000_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_16_1_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_4_10_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vo

In [32]:
with open('/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/exec_all.sh', 'w') as f:
    f.write(sbatch_execs)

In [33]:
cat /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/exec_all.sh

sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_8_100_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_4_100_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_16_1000_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_16_100_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_8_10_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_4_1000_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_16_1_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_4_10_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vo

## 